## Configuration

Select signal to analyze by uncommenting the desired line below.

In [30]:
# ============================================================================
# SIGNAL SELECTION (Uncomment one signal)
# ============================================================================

SELECTED_SIGNAL = "cdx_etf_basis"
# SELECTED_SIGNAL = "cdx_vix_gap"
# SELECTED_SIGNAL = "spread_momentum"

# ============================================================================
# DATA SOURCE CONFIGURATION
# ============================================================================

USE_BLOOMBERG = True  # Set to False to force synthetic data

print(f"Selected Signal: {SELECTED_SIGNAL}")
print(f"Data Source: {'Bloomberg (with fallback)' if USE_BLOOMBERG else 'Synthetic'}")

Selected Signal: cdx_etf_basis
Data Source: Bloomberg (with fallback)


## Setup

In [31]:
import logging
import warnings
from pathlib import Path
from datetime import datetime

import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Configure logging for notebook
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

print("✅ Setup complete")

✅ Setup complete


In [ ]:
# Import aponyx modules
from aponyx.config import (
    DATA_DIR,
    LOGS_DIR,
    SIGNAL_CATALOG_PATH,
    STRATEGY_CATALOG_PATH,
)
from aponyx.data import fetch_cdx, fetch_vix, fetch_etf
from aponyx.data.sources import BloombergSource, FileSource
from aponyx.data.bloomberg_config import list_securities, get_security_spec
from aponyx.models import compute_registered_signals, SignalConfig
from aponyx.models.registry import SignalRegistry
from aponyx.backtest import run_backtest
from aponyx.backtest.registry import StrategyRegistry
from aponyx.evaluation.suitability import (
    evaluate_signal_suitability,
    SuitabilityConfig,
    SuitabilityRegistry,
)
from aponyx.evaluation.performance import (
    analyze_backtest_performance,
    compute_all_metrics,
    PerformanceConfig,
    PerformanceRegistry,
)
from aponyx.visualization import plot_signal, plot_equity_curve
from aponyx.persistence import save_parquet, save_json

print("✅ Imports complete")

✅ Imports complete


---

## Step 1: Data Acquisition

Load market data required by the selected signal. Automatically falls back to synthetic data if Bloomberg cache is unavailable.

In [33]:
# Load signal metadata to determine data requirements
signal_registry = SignalRegistry(SIGNAL_CATALOG_PATH)
signal_metadata = signal_registry.get_metadata(SELECTED_SIGNAL)

print(f"Signal: {signal_metadata.name}")
print(f"Description: {signal_metadata.description}")
print(f"Data Requirements: {signal_metadata.data_requirements}")
print(f"Enabled: {signal_metadata.enabled}")

2025-11-11 22:03:34,643 - aponyx.models.registry - INFO - Loaded signal registry: catalog=C:\Users\ROG3003\PythonProjects\aponyx\src\aponyx\models\signal_catalog.json, signals=3, enabled=3


Signal: cdx_etf_basis
Description: Flow-driven mispricing signal from CDX-ETF basis divergence
Data Requirements: {'cdx': 'spread', 'etf': 'spread'}
Enabled: True


In [34]:
# Detect available data sources
print(f"\n{'='*80}")
print(f"DATA SOURCE DETECTION")
print(f"{'='*80}\n")

cache_bloomberg = DATA_DIR / "cache" / "bloomberg"
cache_file = DATA_DIR / "cache" / "file"

has_bloomberg_cache = cache_bloomberg.exists() and list(cache_bloomberg.glob("*.parquet"))
has_file_cache = cache_file.exists() and list(cache_file.glob("*.parquet"))

print(f"Bloomberg cache: {cache_bloomberg}")
print(f"  Available: {bool(has_bloomberg_cache)}")
if has_bloomberg_cache:
    print(f"  Files: {len(list(cache_bloomberg.glob('*.parquet')))}")

print(f"\nSynthetic cache: {cache_file}")
print(f"  Available: {bool(has_file_cache)}")
if has_file_cache:
    print(f"  Files: {len(list(cache_file.glob('*.parquet')))}")

print(f"\nConfiguration: USE_BLOOMBERG = {USE_BLOOMBERG}")

# Determine data source
if USE_BLOOMBERG and has_bloomberg_cache:
    data_source = "bloomberg"
    print(f"\n✓ Using Bloomberg data from cache")
elif has_file_cache:
    data_source = "file"
    print(f"\n✓ Using synthetic data from file cache")
    print("  (Run generate_synthetic_data.py if files are missing)")
else:
    raise FileNotFoundError(
        "No data cache found. Please run either:\n"
        "  1. 01_data_download.ipynb (Bloomberg Terminal), or\n"
        "  2. python generate_synthetic_data.py (synthetic data)"
    )


DATA SOURCE DETECTION

Bloomberg cache: C:\Users\ROG3003\PythonProjects\aponyx\data\cache\bloomberg
  Available: False

Synthetic cache: C:\Users\ROG3003\PythonProjects\aponyx\data\cache\file
  Available: True
  Files: 8

Configuration: USE_BLOOMBERG = True

✓ Using synthetic data from file cache
  (Run generate_synthetic_data.py if files are missing)


In [35]:
# Load market data from cache
print(f"\n{'='*80}")
print(f"LOADING MARKET DATA")
print(f"{'='*80}\n")

use_cache = True

if data_source == "bloomberg":
    # Bloomberg cache available
    from aponyx.data.sources import BloombergSource
    print("Data source: Bloomberg cache\n")
    
    source = BloombergSource()
    cdx_df = fetch_cdx(source=source, security="cdx_ig_5y", use_cache=use_cache)
    vix_df = fetch_vix(source=source, use_cache=use_cache)
    etf_df = fetch_etf(source=source, security="hyg", use_cache=use_cache)
    
elif data_source == "file":
    # Synthetic data cache available
    from aponyx.data.sources import FileSource
    print("Data source: Synthetic data cache")
    print("  (Run generate_synthetic_data.py if files are missing)\n")
    
    cdx_source = FileSource(cache_file / "cdx_cdx_ig_5y.parquet")
    vix_source = FileSource(cache_file / "vix_vix.parquet")
    etf_source = FileSource(cache_file / "etf_hyg.parquet")
    
    cdx_df = fetch_cdx(source=cdx_source, security="cdx_ig_5y", use_cache=use_cache)
    vix_df = fetch_vix(source=vix_source, use_cache=use_cache)
    etf_df = fetch_etf(source=etf_source, security="hyg", use_cache=use_cache)

# Load and verify CDX IG 5Y data
print(f"Loading CDX IG 5Y...")
print(f"✓ Loaded CDX IG 5Y: {len(cdx_df)} rows")
print(f"  Columns: {list(cdx_df.columns)}")
print(f"  Date range: {cdx_df.index.min()} to {cdx_df.index.max()}")

if 'spread' not in cdx_df.columns:
    raise ValueError(f"CDX data missing 'spread' column. Found: {list(cdx_df.columns)}")

print()

# Load and verify VIX data
print("Loading VIX...")
print(f"✓ Loaded VIX: {len(vix_df)} rows")
print(f"  Columns: {list(vix_df.columns)}")
print(f"  Date range: {vix_df.index.min()} to {vix_df.index.max()}")

if 'level' not in vix_df.columns:
    raise ValueError(f"VIX data missing 'level' column. Found: {list(vix_df.columns)}")

print()

# Load and verify ETF (HYG) data
print("Loading HYG ETF...")
print(f"✓ Loaded HYG ETF: {len(etf_df)} rows")
print(f"  Columns: {list(etf_df.columns)}")
print(f"  Date range: {etf_df.index.min()} to {etf_df.index.max()}")

if 'spread' not in etf_df.columns:
    raise ValueError(f"ETF data missing 'spread' column. Found: {list(etf_df.columns)}")

print()

# Create market data dictionary
market_data = {
    "cdx": cdx_df,
    "vix": vix_df,
    "etf": etf_df,
}

2025-11-11 22:03:34,669 - aponyx.data.fetch - INFO - Fetching CDX from file
2025-11-11 22:03:34,669 - aponyx.data.providers.file - INFO - Fetching cdx from file: C:\Users\ROG3003\PythonProjects\aponyx\data\cache\file\cdx_cdx_ig_5y.parquet
2025-11-11 22:03:34,670 - aponyx.persistence.parquet_io - INFO - Loading Parquet file: path=C:\Users\ROG3003\PythonProjects\aponyx\data\cache\file\cdx_cdx_ig_5y.parquet, columns=all
2025-11-11 22:03:34,669 - aponyx.data.providers.file - INFO - Fetching cdx from file: C:\Users\ROG3003\PythonProjects\aponyx\data\cache\file\cdx_cdx_ig_5y.parquet
2025-11-11 22:03:34,670 - aponyx.persistence.parquet_io - INFO - Loading Parquet file: path=C:\Users\ROG3003\PythonProjects\aponyx\data\cache\file\cdx_cdx_ig_5y.parquet, columns=all
2025-11-11 22:03:34,684 - aponyx.persistence.parquet_io - INFO - Loaded 1304 rows, 2 columns from C:\Users\ROG3003\PythonProjects\aponyx\data\cache\file\cdx_cdx_ig_5y.parquet
2025-11-11 22:03:34,685 - aponyx.data.providers.file - INFO


LOADING MARKET DATA

Data source: Synthetic data cache
  (Run generate_synthetic_data.py if files are missing)



2025-11-11 22:03:34,783 - aponyx.data.registry - INFO - Registered dataset: name=cache_vix_d09015690dfa93d9, instrument=vix, rows=1304
2025-11-11 22:03:34,784 - aponyx.data.fetch - INFO - Fetched VIX data: 1304 rows, 2020-11-11 00:00:00 to 2024-06-06 00:00:00
2025-11-11 22:03:34,785 - aponyx.data.fetch - INFO - Fetching ETF from file
2025-11-11 22:03:34,786 - aponyx.data.providers.file - INFO - Fetching etf from file: C:\Users\ROG3003\PythonProjects\aponyx\data\cache\file\etf_hyg.parquet
2025-11-11 22:03:34,787 - aponyx.persistence.parquet_io - INFO - Loading Parquet file: path=C:\Users\ROG3003\PythonProjects\aponyx\data\cache\file\etf_hyg.parquet, columns=all
2025-11-11 22:03:34,784 - aponyx.data.fetch - INFO - Fetched VIX data: 1304 rows, 2020-11-11 00:00:00 to 2024-06-06 00:00:00
2025-11-11 22:03:34,785 - aponyx.data.fetch - INFO - Fetching ETF from file
2025-11-11 22:03:34,786 - aponyx.data.providers.file - INFO - Fetching etf from file: C:\Users\ROG3003\PythonProjects\aponyx\data\

Loading CDX IG 5Y...
✓ Loaded CDX IG 5Y: 1304 rows
  Columns: ['spread', 'security']
  Date range: 2020-11-11 00:00:00 to 2024-06-06 00:00:00

Loading VIX...
✓ Loaded VIX: 1304 rows
  Columns: ['level']
  Date range: 2020-11-11 00:00:00 to 2024-06-06 00:00:00

Loading HYG ETF...
✓ Loaded HYG ETF: 1304 rows
  Columns: ['spread', 'security']
  Date range: 2020-11-11 00:00:00 to 2024-06-06 00:00:00



In [36]:
# Display data summary
summary_data = [
    {
        'Dataset': 'CDX IG 5Y',
        'Rows': len(cdx_df),
        'Start': cdx_df.index.min().strftime('%Y-%m-%d'),
        'End': cdx_df.index.max().strftime('%Y-%m-%d'),
        'Columns': ', '.join(cdx_df.columns),
    },
    {
        'Dataset': 'VIX',
        'Rows': len(vix_df),
        'Start': vix_df.index.min().strftime('%Y-%m-%d'),
        'End': vix_df.index.max().strftime('%Y-%m-%d'),
        'Columns': ', '.join(vix_df.columns),
    },
    {
        'Dataset': 'HYG ETF',
        'Rows': len(etf_df),
        'Start': etf_df.index.min().strftime('%Y-%m-%d'),
        'End': etf_df.index.max().strftime('%Y-%m-%d'),
        'Columns': ', '.join(etf_df.columns),
    },
]

summary_df = pd.DataFrame(summary_data)
print("Data Summary:\n")
print(summary_df.to_markdown(index=False))
print(f"\n✓ Loaded {len(market_data)} datasets for signal computation")

Data Summary:

| Dataset   |   Rows | Start      | End        | Columns          |
|:----------|-------:|:-----------|:-----------|:-----------------|
| CDX IG 5Y |   1304 | 2020-11-11 | 2024-06-06 | spread, security |
| VIX       |   1304 | 2020-11-11 | 2024-06-06 | level            |
| HYG ETF   |   1304 | 2020-11-11 | 2024-06-06 | spread, security |

✓ Loaded 3 datasets for signal computation


---

## Step 2: Signal Computation

Compute the selected signal using the signal catalog framework.

In [37]:
# Create signal configuration
signal_config = SignalConfig(
    lookback=20,
    min_periods=10,
)

print(f"\n{'='*80}")
print(f"COMPUTING SIGNAL: {SELECTED_SIGNAL}")
print(f"{'='*80}\n")

print(f"Configuration:")
print(f"  Lookback window: {signal_config.lookback} days")
print(f"  Minimum periods: {signal_config.min_periods} observations")
print(f"  Selected signal: {SELECTED_SIGNAL}")

# Compute all enabled signals (registry manages which are enabled)
signals_dict = compute_registered_signals(
    registry=signal_registry,
    market_data=market_data,
    config=signal_config,
)

# Extract the selected signal
if SELECTED_SIGNAL not in signals_dict:
    available = list(signals_dict.keys())
    raise ValueError(
        f"Signal '{SELECTED_SIGNAL}' not found in computed signals.\n"
        f"Available signals: {available}\n"
        f"Make sure the signal is enabled in {SIGNAL_CATALOG_PATH}"
    )

signal = signals_dict[SELECTED_SIGNAL]

print(f"\n✓ Computed signal: {SELECTED_SIGNAL}")
print(f"   Valid observations: {signal.notna().sum()} / {len(signal)}")
print(f"   Date range: {signal.index.min()} to {signal.index.max()}")

2025-11-11 22:03:34,873 - aponyx.models.catalog - INFO - Computing 3 enabled signals: cdx_etf_basis, cdx_vix_gap, spread_momentum
2025-11-11 22:03:34,874 - aponyx.models.signals - INFO - Computing CDX-ETF basis: cdx_rows=1304, etf_rows=1304, lookback=20
2025-11-11 22:03:34,876 - aponyx.models.signals - INFO - Computing CDX-VIX gap: cdx_rows=1304, vix_rows=1304, lookback=20
2025-11-11 22:03:34,877 - aponyx.models.signals - INFO - Computing spread momentum: cdx_rows=1304, lookback=20
2025-11-11 22:03:34,879 - aponyx.models.catalog - INFO - Successfully computed 3 signals



COMPUTING SIGNAL: cdx_etf_basis

Configuration:
  Lookback window: 20 days
  Minimum periods: 10 observations
  Selected signal: cdx_etf_basis

✓ Computed signal: cdx_etf_basis
   Valid observations: 1295 / 1304
   Date range: 2020-11-11 00:00:00 to 2024-06-06 00:00:00


In [38]:
# Visualize signal
fig = plot_signal(signal, title=f"Signal: {SELECTED_SIGNAL}")

# Add threshold reference lines
fig.add_hline(y=1.5, line_dash="dash", line_color="green", annotation_text="Entry (+1.5)")
fig.add_hline(y=-1.5, line_dash="dash", line_color="red", annotation_text="Entry (-1.5)")
fig.add_hline(y=0.75, line_dash="dot", line_color="lightgreen", annotation_text="Exit (+0.75)")
fig.add_hline(y=-0.75, line_dash="dot", line_color="lightcoral", annotation_text="Exit (-0.75)")

fig.show()

2025-11-11 22:03:34,886 - aponyx.visualization.plots - INFO - Plotting signal: 1304 observations


In [39]:
# Display signal statistics
stats = {
    "Metric": [
        "Count",
        "Mean",
        "Std Dev",
        "Min",
        "25%",
        "50%",
        "75%",
        "Max",
        "Autocorr (lag-1)",
    ],
    "Value": [
        f"{signal.notna().sum()}",
        f"{signal.mean():.4f}",
        f"{signal.std():.4f}",
        f"{signal.min():.4f}",
        f"{signal.quantile(0.25):.4f}",
        f"{signal.median():.4f}",
        f"{signal.quantile(0.75):.4f}",
        f"{signal.max():.4f}",
        f"{signal.autocorr(lag=1):.4f}",
    ],
}

stats_df = pd.DataFrame(stats)
print("\n**Signal Statistics**\n")
print(stats_df.to_markdown(index=False))


**Signal Statistics**

| Metric           |     Value |
|:-----------------|----------:|
| Count            | 1295      |
| Mean             |   -0.1359 |
| Std Dev          |    1.2872 |
| Min              |   -3.3257 |
| 25%              |   -1.2039 |
| 50%              |   -0.0282 |
| 75%              |    0.9028 |
| Max              |    2.6169 |
| Autocorr (lag-1) |    0.8538 |


---

## Step 3: Suitability Evaluation

Evaluate signal-product suitability using 4-component scoring framework.

In [40]:
# Prepare target data (CDX spread changes)
cdx_spread = market_data["cdx"]["spread"]

# Evaluate suitability
suitability_config = SuitabilityConfig()

suitability_result = evaluate_signal_suitability(
    signal=signal,
    target_change=cdx_spread,
    config=suitability_config,
)

print(f"\n✅ Suitability evaluation complete")
print(f"   Decision: {suitability_result.decision}")
print(f"   Composite Score: {suitability_result.composite_score:.4f}")

2025-11-11 22:03:34,967 - aponyx.evaluation.suitability.evaluator - INFO - Starting suitability evaluation: signal=spread, config=SuitabilityConfig(lags=[1, 3, 5], min_obs=500, pass_threshold=0.7, hold_threshold=0.4, data_health_weight=0.2, predictive_weight=0.4, economic_weight=0.2, stability_weight=0.2)
2025-11-11 22:03:34,969 - aponyx.evaluation.suitability.evaluator - INFO - Data health score: 0.965
2025-11-11 22:03:34,969 - aponyx.evaluation.suitability.evaluator - INFO - Data health score: 0.965
2025-11-11 22:03:34,976 - aponyx.evaluation.suitability.evaluator - INFO - Predictive score: 0.697 (mean |t-stat|=2.092 across 3 lags)
2025-11-11 22:03:34,976 - aponyx.evaluation.suitability.evaluator - INFO - Economic score: 0.600
2025-11-11 22:03:34,979 - aponyx.evaluation.suitability.evaluator - INFO - Stability score: 0.000
2025-11-11 22:03:34,979 - aponyx.evaluation.suitability.evaluator - INFO - Composite score: 0.592
2025-11-11 22:03:34,979 - aponyx.evaluation.suitability.evaluator


✅ Suitability evaluation complete
   Decision: HOLD
   Composite Score: 0.5920


In [41]:
# Display component scores
component_data = [
    {
        "Component": "Data Health",
        "Weight": "20%",
        "Score": f"{suitability_result.data_health_score:.4f}",
        "Weighted": f"{suitability_result.data_health_score * 0.2:.4f}",
    },
    {
        "Component": "Predictive",
        "Weight": "40%",
        "Score": f"{suitability_result.predictive_score:.4f}",
        "Weighted": f"{suitability_result.predictive_score * 0.4:.4f}",
    },
    {
        "Component": "Economic",
        "Weight": "20%",
        "Score": f"{suitability_result.economic_score:.4f}",
        "Weighted": f"{suitability_result.economic_score * 0.2:.4f}",
    },
    {
        "Component": "Stability",
        "Weight": "20%",
        "Score": f"{suitability_result.stability_score:.4f}",
        "Weighted": f"{suitability_result.stability_score * 0.2:.4f}",
    },
    {
        "Component": "**Composite**",
        "Weight": "**100%**",
        "Score": "",
        "Weighted": f"**{suitability_result.composite_score:.4f}**",
    },
]

component_df = pd.DataFrame(component_data)
print("\n**Suitability Evaluation**\n")
print(component_df.to_markdown(index=False))
print(f"\n**Decision: {suitability_result.decision}**")


**Suitability Evaluation**

| Component     | Weight   | Score   | Weighted   |
|:--------------|:---------|:--------|:-----------|
| Data Health   | 20%      | 0.9655  | 0.1931     |
| Predictive    | 40%      | 0.6973  | 0.2789     |
| Economic      | 20%      | 0.6000  | 0.1200     |
| Stability     | 20%      | 0.0000  | 0.0000     |
| **Composite** | **100%** |         | **0.5920** |

**Decision: HOLD**


In [42]:
# Register suitability result
from aponyx.evaluation.suitability.report import generate_suitability_report, save_report
from aponyx.config import EVALUATION_DIR, SUITABILITY_REGISTRY_PATH

suitability_registry = SuitabilityRegistry(SUITABILITY_REGISTRY_PATH)

# Generate report
report_content = generate_suitability_report(
    result=suitability_result,
    signal_id=SELECTED_SIGNAL,
    product_id="cdx_ig_5y",
)

# Save report
report_path = save_report(
    report=report_content,
    signal_id=SELECTED_SIGNAL,
    product_id="cdx_ig_5y",
    output_dir=EVALUATION_DIR,
)

# Register evaluation
suitability_registry.register_evaluation(
    result=suitability_result,
    signal_id=SELECTED_SIGNAL,
    product_id="cdx_ig_5y",
)

print(f"\n✅ Suitability report saved: {report_path}")
print(f"✅ Suitability result registered")

2025-11-11 22:03:34,997 - aponyx.persistence.json_io - INFO - Loading JSON from C:\Users\ROG3003\PythonProjects\aponyx\src\aponyx\evaluation\suitability\suitability_registry.json
2025-11-11 22:03:35,007 - aponyx.evaluation.suitability.registry - INFO - Loaded existing registry: 1 evaluations
2025-11-11 22:03:35,008 - aponyx.evaluation.suitability.report - INFO - Saved report to C:\Users\ROG3003\PythonProjects\aponyx\reports\suitability\cdx_etf_basis_cdx_ig_5y_20251111_220335.md
2025-11-11 22:03:35,010 - aponyx.persistence.json_io - INFO - Saving JSON to C:\Users\ROG3003\PythonProjects\aponyx\src\aponyx\evaluation\suitability\suitability_registry.json (2 top-level keys)
2025-11-11 22:03:35,011 - aponyx.evaluation.suitability.registry - INFO - Registered evaluation: cdx_etf_basis_cdx_ig_5y_20251111_220335 (decision=HOLD, score=0.592)



✅ Suitability report saved: C:\Users\ROG3003\PythonProjects\aponyx\reports\suitability\cdx_etf_basis_cdx_ig_5y_20251111_220335.md
✅ Suitability result registered


---

## Step 4: Multi-Strategy Backtest

Run backtests for all enabled strategies from the strategy catalog.

In [43]:
# Load strategy registry
strategy_registry = StrategyRegistry(STRATEGY_CATALOG_PATH)

# Get all enabled strategies
enabled_strategies = strategy_registry.get_enabled()

print(f"Found {len(enabled_strategies)} enabled strategies:")
for name in enabled_strategies:
    print(f"  - {name}")

2025-11-11 22:03:35,022 - aponyx.backtest.registry - INFO - Loaded strategy registry: catalog=C:\Users\ROG3003\PythonProjects\aponyx\src\aponyx\backtest\strategy_catalog.json, strategies=4, enabled=3


Found 3 enabled strategies:
  - conservative
  - balanced
  - aggressive


In [44]:
# Run backtests for all strategies
backtest_results = {}
performance_metrics = {}
failed_strategies = []

for strategy_name, strategy_metadata in enabled_strategies.items():
    try:
        print(f"\nRunning backtest: {strategy_name}")
        
        # Convert metadata to config
        config = strategy_metadata.to_config()
        
        # Run backtest
        result = run_backtest(
            composite_signal=signal,
            spread=cdx_spread,
            config=config,
        )
        
        # Compute basic metrics for sorting
        metrics = compute_performance_metrics(
            result.pnl,
            result.positions,
        )
        
        backtest_results[strategy_name] = result
        performance_metrics[strategy_name] = metrics
        
        print(f"  ✅ Sharpe: {metrics.sharpe_ratio:.4f}, Trades: {metrics.n_trades}")
        
    except Exception as e:
        logger.warning(f"Backtest failed for {strategy_name}: {e}")
        failed_strategies.append(strategy_name)
        print(f"  ❌ Failed: {e}")
        continue

print(f"\n✅ Backtests complete: {len(backtest_results)} successful, {len(failed_strategies)} failed")

2025-11-11 22:03:35,028 - aponyx.backtest.engine - INFO - Starting backtest: dates=1304, entry_threshold=2.00, position_size=10.0MM



Running backtest: conservative


2025-11-11 22:03:35,067 - aponyx.backtest.engine - INFO - Backtest complete: trades=53, total_pnl=$-11748867, avg_per_trade=$-221677
2025-11-11 22:03:35,067 - aponyx.backtest.metrics - INFO - Computing performance metrics
2025-11-11 22:03:35,071 - aponyx.backtest.metrics - INFO - Metrics computed: sharpe=-0.46, max_dd=$-43049117, hit_rate=37.7%
2025-11-11 22:03:35,072 - aponyx.backtest.engine - INFO - Starting backtest: dates=1304, entry_threshold=1.50, position_size=10.0MM
2025-11-11 22:03:35,067 - aponyx.backtest.metrics - INFO - Computing performance metrics
2025-11-11 22:03:35,071 - aponyx.backtest.metrics - INFO - Metrics computed: sharpe=-0.46, max_dd=$-43049117, hit_rate=37.7%
2025-11-11 22:03:35,072 - aponyx.backtest.engine - INFO - Starting backtest: dates=1304, entry_threshold=1.50, position_size=10.0MM
2025-11-11 22:03:35,109 - aponyx.backtest.engine - INFO - Backtest complete: trades=87, total_pnl=$-9987160, avg_per_trade=$-114795
2025-11-11 22:03:35,110 - aponyx.backtest.m

  ✅ Sharpe: -0.4647, Trades: 53

Running backtest: balanced
  ✅ Sharpe: -0.3206, Trades: 87

Running backtest: aggressive


2025-11-11 22:03:35,152 - aponyx.backtest.engine - INFO - Backtest complete: trades=110, total_pnl=$-44948953, avg_per_trade=$-408627
2025-11-11 22:03:35,155 - aponyx.backtest.metrics - INFO - Computing performance metrics
2025-11-11 22:03:35,159 - aponyx.backtest.metrics - INFO - Metrics computed: sharpe=-1.31, max_dd=$-61295928, hit_rate=40.9%


  ✅ Sharpe: -1.3139, Trades: 110

✅ Backtests complete: 3 successful, 0 failed


In [45]:
# Save consolidated backtest results
if backtest_results:
    # Combine all P&L DataFrames
    all_pnl = {}
    for strategy_name, result in backtest_results.items():
        all_pnl[strategy_name] = result.pnl
    
    # Save to parquet (overwrites previous run)
    output_path = DATA_DIR / "processed" / f"backtest_{SELECTED_SIGNAL}_all_strategies.parquet"
    
    # Create multi-level DataFrame
    combined_pnl = pd.concat(all_pnl, names=["strategy", "date"])
    save_parquet(combined_pnl.reset_index(), output_path)
    
    print(f"\n✅ Backtest results saved: {output_path}")
else:
    print("\n❌ No successful backtests to save")

2025-11-11 22:03:35,168 - aponyx.persistence.parquet_io - INFO - Saving DataFrame to Parquet: path=C:\Users\ROG3003\PythonProjects\aponyx\data\processed\backtest_cdx_etf_basis_all_strategies.parquet, rows=3885, columns=6, compression=snappy



✅ Backtest results saved: C:\Users\ROG3003\PythonProjects\aponyx\data\processed\backtest_cdx_etf_basis_all_strategies.parquet



In [46]:
# Sort strategies by Sharpe ratio
if performance_metrics:
    sorted_strategies = sorted(
        performance_metrics.items(),
        key=lambda x: x[1].sharpe_ratio,
        reverse=True,
    )
    
    print("\n**Strategies by Sharpe Ratio**\n")
    for i, (strategy_name, metrics) in enumerate(sorted_strategies, 1):
        print(f"{i}. {strategy_name}: {metrics.sharpe_ratio:.4f}")
else:
    sorted_strategies = []
    print("\n❌ No successful backtests to rank")


**Strategies by Sharpe Ratio**

1. balanced: -0.3206
2. conservative: -0.4647
3. aggressive: -1.3139


In [47]:
# Visualize equity curves (single column layout)
if sorted_strategies:
    n_strategies = len(sorted_strategies)
    
    # Create subplots in single column
    fig = make_subplots(
        rows=n_strategies,
        cols=1,
        subplot_titles=[f"{name} (Sharpe: {metrics.sharpe_ratio:.2f})" 
                       for name, metrics in sorted_strategies],
        vertical_spacing=0.05,
        shared_xaxes=True,
    )
    
    # Add equity curve for each strategy
    for i, (strategy_name, metrics) in enumerate(sorted_strategies, 1):
        result = backtest_results[strategy_name]
        
        fig.add_trace(
            go.Scatter(
                x=result.pnl.index,
                y=result.pnl["cumulative_pnl"],
                mode="lines",
                name=strategy_name,
                line=dict(width=2),
                showlegend=False,
            ),
            row=i,
            col=1,
        )
        
        # Update y-axis label
        fig.update_yaxes(title_text="Cumulative P&L", row=i, col=1)
    
    # Update layout
    fig.update_layout(
        height=300 * n_strategies,
        title_text=f"Equity Curves: {SELECTED_SIGNAL} (Sorted by Sharpe)",
        showlegend=False,
    )
    
    fig.update_xaxes(title_text="Date", row=n_strategies, col=1)
    
    fig.show()
else:
    print("❌ No equity curves to display")

---

## Step 5: Performance Analysis

Comprehensive post-backtest evaluation with extended metrics and attribution.

In [48]:
# Analyze performance for all successful backtests
from aponyx.evaluation.performance.report import (
    generate_performance_report,
    save_report as save_perf_report,
)
from aponyx.config import PERFORMANCE_REPORTS_DIR, PERFORMANCE_REGISTRY_PATH

performance_results = {}
performance_registry = PerformanceRegistry(PERFORMANCE_REGISTRY_PATH)
performance_config = PerformanceConfig()

for strategy_name, backtest_result in backtest_results.items():
    print(f"\nAnalyzing performance: {strategy_name}")
    
    # Analyze performance
    perf_result = analyze_backtest_performance(
        backtest_result=backtest_result,
        config=performance_config,
    )
    
    performance_results[strategy_name] = perf_result
    
    # Generate report
    report_content = generate_performance_report(
        result=perf_result,
        signal_id=SELECTED_SIGNAL,
        strategy_id=strategy_name,
    )
    
    # Save report
    report_path = save_perf_report(
        report=report_content,
        signal_id=SELECTED_SIGNAL,
        strategy_id=strategy_name,
        output_dir=PERFORMANCE_REPORTS_DIR,
    )
    
    # Register result
    performance_registry.register_evaluation(
        result=perf_result,
        signal_id=SELECTED_SIGNAL,
        strategy_id=strategy_name,
    )
    
    print(f"  ✅ Report saved: {report_path}")

print(f"\n✅ Performance analysis complete for {len(performance_results)} strategies")

2025-11-11 22:03:35,236 - aponyx.persistence.json_io - INFO - Loading JSON from C:\Users\ROG3003\PythonProjects\aponyx\src\aponyx\evaluation\performance\performance_registry.json
2025-11-11 22:03:35,246 - aponyx.evaluation.performance.registry - INFO - Loaded existing performance registry: 3 evaluations
2025-11-11 22:03:35,248 - aponyx.evaluation.performance.analyzer - INFO - Analyzing backtest performance: config=PerformanceConfig(min_obs=252, n_subperiods=4, risk_free_rate=0.0, rolling_window=63, report_format='markdown', attribution_quantiles=3)
2025-11-11 22:03:35,249 - aponyx.evaluation.performance.risk_metrics - INFO - Computing extended risk metrics: window=63 days
2025-11-11 22:03:35,246 - aponyx.evaluation.performance.registry - INFO - Loaded existing performance registry: 3 evaluations
2025-11-11 22:03:35,248 - aponyx.evaluation.performance.analyzer - INFO - Analyzing backtest performance: config=PerformanceConfig(min_obs=252, n_subperiods=4, risk_free_rate=0.0, rolling_windo


Analyzing performance: conservative


2025-11-11 22:03:35,262 - aponyx.evaluation.performance.risk_metrics - INFO - Extended metrics computed: profit_factor=0.86, tail_ratio=0.94, consistency=36.9%
2025-11-11 22:03:35,263 - aponyx.evaluation.performance.decomposition - INFO - Computing return attribution: n_quantiles=3
2025-11-11 22:03:35,267 - aponyx.evaluation.performance.decomposition - INFO - Attribution computed: long=-0.1%, wins=444.4%
2025-11-11 22:03:35,268 - aponyx.evaluation.performance.analyzer - INFO - Performance evaluation complete: stability=0.50, profit_factor=0.86
2025-11-11 22:03:35,269 - aponyx.evaluation.performance.report - INFO - Saved performance report to C:\Users\ROG3003\PythonProjects\aponyx\reports\performance\cdx_etf_basis_conservative_20251111_220335.md
2025-11-11 22:03:35,270 - aponyx.persistence.json_io - INFO - Saving JSON to C:\Users\ROG3003\PythonProjects\aponyx\src\aponyx\evaluation\performance\performance_registry.json (4 top-level keys)
2025-11-11 22:03:35,272 - aponyx.evaluation.perfor

  ✅ Report saved: C:\Users\ROG3003\PythonProjects\aponyx\reports\performance\cdx_etf_basis_conservative_20251111_220335.md

Analyzing performance: balanced
  ✅ Report saved: C:\Users\ROG3003\PythonProjects\aponyx\reports\performance\cdx_etf_basis_balanced_20251111_220335.md

Analyzing performance: aggressive
  ✅ Report saved: C:\Users\ROG3003\PythonProjects\aponyx\reports\performance\cdx_etf_basis_aggressive_20251111_220335.md

✅ Performance analysis complete for 3 strategies


In [49]:
# Create comparative metrics table
if performance_results:
    metrics_data = []
    
    for strategy_name, perf_result in performance_results.items():
        # Extended metrics from performance evaluation
        ext = perf_result.metrics
        # Basic metrics from backtest
        basic = performance_metrics[strategy_name]
        
        metrics_data.append({
            "Strategy": strategy_name,
            "Sharpe": f"{basic.sharpe_ratio:.4f}",
            "Sortino": f"{basic.sortino_ratio:.4f}",
            "Max DD": f"{basic.max_drawdown:.2%}",
            "Profit Factor": f"{ext['profit_factor']:.4f}",
            "Tail Ratio": f"{ext['tail_ratio']:.4f}",
            "Consistency": f"{ext['consistency_score']:.2%}",
            "Trades": f"{basic.n_trades}",
        })
    
    # Sort by Sharpe
    metrics_df = pd.DataFrame(metrics_data)
    metrics_df = metrics_df.sort_values(
        by="Sharpe",
        key=lambda x: x.astype(float),
        ascending=False,
    )
    
    print("\n**Performance Metrics Comparison**\n")
    print(metrics_df.to_markdown(index=False))
else:
    print("\n❌ No performance results to display")


**Performance Metrics Comparison**

| Strategy     |   Sharpe |   Sortino | Max DD          |   Profit Factor |   Tail Ratio | Consistency   |   Trades |
|:-------------|---------:|----------:|:----------------|----------------:|-------------:|:--------------|---------:|
| balanced     |  -0.3206 |   -0.3322 | -4009716230.08% |          0.9251 |       1.0795 | 42.43%        |       87 |
| conservative |  -0.4647 |   -0.3336 | -4304911666.90% |          0.8577 |       0.9402 | 36.86%        |       53 |
| aggressive   |  -1.3139 |   -1.4894 | -6129592761.17% |          0.7647 |       0.7755 | 42.59%        |      110 |


In [50]:
# Display attribution for top performer
if sorted_strategies:
    top_strategy = sorted_strategies[0][0]
    top_result = performance_results[top_strategy]
    
    print(f"\n**Attribution Analysis: {top_strategy} (Top Performer)**\n")
    
    # Directional attribution
    print("**Directional Attribution:**")
    dir_attr = top_result.attribution["direction"]
    total_pnl = dir_attr['long_pnl'] + dir_attr['short_pnl']
    print(f"  Long P&L: {dir_attr['long_pnl']:.2f} ({dir_attr['long_pct']:.1%})")
    print(f"  Short P&L: {dir_attr['short_pnl']:.2f} ({dir_attr['short_pct']:.1%})")
    print(f"  Total: {total_pnl:.2f}")
    
    # Signal strength attribution
    print("\n**Signal Strength Attribution (Quantiles):**")
    sig_attr = top_result.attribution["signal_strength"]
    n_quantiles = top_result.config.attribution_quantiles
    for i in range(1, n_quantiles + 1):
        pnl = sig_attr[f'q{i}_pnl']
        pct = sig_attr[f'q{i}_pct']
        print(f"  Q{i}: {pnl:.2f} ({pct:.1%})")
    
    # Win/loss attribution
    print("\n**Win/Loss Attribution:**")
    wl_attr = top_result.attribution["win_loss"]
    print(f"  Wins: {wl_attr['gross_wins']:.2f} ({wl_attr['win_contribution']:.1%})")
    print(f"  Losses: {wl_attr['gross_losses']:.2f} ({wl_attr['loss_contribution']:.1%})")
    # Calculate win rate from wins/losses if not provided
    n_wins = wl_attr.get('n_wins', 0)
    n_losses = wl_attr.get('n_losses', 0)
    total_trades = n_wins + n_losses
    win_rate = n_wins / total_trades if total_trades > 0 else 0.0
    print(f"  Win Rate: {win_rate:.1%}")
else:
    print("\n❌ No attribution to display")


**Attribution Analysis: balanced (Top Performer)**

**Directional Attribution:**
  Long P&L: -14749824.15 (112.5%)
  Short P&L: 1644674.53 (-12.5%)
  Total: -13105149.62

**Signal Strength Attribution (Quantiles):**
  Q1: -7418390.74 (56.6%)
  Q2: -4018320.62 (30.7%)
  Q3: -1668438.26 (12.7%)

**Win/Loss Attribution:**
  Wins: 104202940.43 (795.1%)
  Losses: -117308090.04 (895.1%)
  Win Rate: 0.0%


---

## Workflow Complete

Successfully completed single-signal research workflow.

### What Was Accomplished

✅ **Data Loaded** — Acquired required market data with automatic Bloomberg/synthetic fallback

✅ **Signal Computed** — Generated z-score normalized signal from market data

✅ **Suitability Evaluated** — 4-component quality screening with PASS/HOLD/FAIL decision

✅ **Backtests Executed** — Tested signal across all enabled strategies from catalog

✅ **Performance Analyzed** — Comprehensive evaluation with extended metrics and attribution

### Key Outputs

**Suitability Report:**
```
reports/suitability/{signal_id}_cdx_ig_5y_{timestamp}.md
```

**Consolidated Backtest Results:**
```
data/processed/backtest_{signal_id}_all_strategies.parquet
```

**Performance Reports:**
```
reports/performance/{signal_id}_{strategy_id}_{timestamp}.md
```

### Re-Running This Notebook

- **Same signal, different data:** Update `USE_BLOOMBERG` toggle or refresh cache
- **Different signal:** Change `SELECTED_SIGNAL` in configuration cell
- **Different strategies:** Edit `src/aponyx/backtest/strategy_catalog.json`
- **Overwrites:** Backtest results file overwrites previous run for same signal

### Troubleshooting

**Bloomberg Terminal Connection Failed:**
- Ensure Bloomberg Terminal is running and logged in
- Check that Bloomberg data was previously cached via `01_data_download.ipynb`
- Set `USE_BLOOMBERG = False` to use synthetic data

**No Cache Available:**
- Run `01_data_download.ipynb` to cache Bloomberg data, or
- Run `python src/aponyx/notebooks/generate_synthetic_data.py` to create test data

**Strategy Backtest Failed:**
- Check strategy configuration in `strategy_catalog.json`
- Ensure entry_threshold > exit_threshold
- Review error message in cell output
- Failed strategies are automatically skipped (notebook continues)

### Testing New Signals

To research a new signal idea:

1. **Implement compute function** in `src/aponyx/models/signals.py`:
   ```python
   def compute_new_signal(
       cdx_df: pd.DataFrame,
       config: SignalConfig | None = None,
   ) -> pd.Series:
       """Compute new signal logic here."""
       # Implementation...
       return signal  # Positive = long credit risk
   ```

2. **Register in catalog** at `src/aponyx/models/signal_catalog.json`:
   ```json
   {
     "name": "new_signal",
     "description": "Description of signal logic",
     "compute_function_name": "compute_new_signal",
     "data_requirements": {"cdx": "spread"},
     "arg_mapping": ["cdx"],
     "enabled": true
   }
   ```

3. **Update configuration cell** above:
   ```python
   SELECTED_SIGNAL = "new_signal"
   ```

4. **Re-run notebook** to evaluate new signal

### Strategy Customization

To test different entry/exit thresholds:

1. **Edit strategy catalog** at `src/aponyx/backtest/strategy_catalog.json`:
   ```json
   {
     "name": "custom_strategy",
     "description": "Custom threshold configuration",
     "entry_threshold": 1.2,
     "exit_threshold": 0.6,
     "enabled": true
   }
   ```

2. **Re-run backtest section** — New strategy will be automatically included

---